In [1]:
# # ===============================
# # ✅ 必要套件安裝 (請在終端機執行)
# # ===============================
# # pip install transformers datasets scikit-learn pandas torch torchvision torchaudio matplotlib seaborn openpyxl

# import pandas as pd
# import os
# import torch
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from datasets import Dataset
# from transformers import (
#     BertTokenizerFast,
#     BertForSequenceClassification,
#     TrainingArguments,
#     Trainer,
#     EarlyStoppingCallback,
#     TextClassificationPipeline
# )
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support
# import matplotlib.font_manager as fm # For font checking/setting

# # ===============================
# # ✅ 路徑設定 (請根據你的本機資料夾結構修改)
# # ===============================
# DATA_DIR = "./data/" # 存放 CSV 檔案的資料夾
# OUTPUT_DIR = "./output/" # 存放模型、結果、圖片的資料夾

# # 確保輸出資料夾存在
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR, "bert_results"), exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR, "bert_logs"), exist_ok=True)
# os.makedirs(os.path.join(OUTPUT_DIR, "bert_model_chinese"), exist_ok=True)

# # CSV 檔案路徑
# FILE_DF1 = os.path.join(DATA_DIR, 'finfo_posts_產險_壽險_投資型.csv')
# FILE_DF2 = os.path.join(DATA_DIR, 'mobile01_處理後.csv')
# FILE_DF3 = os.path.join(DATA_DIR, 'ptt_語料_處理後.csv')

# # 輸出檔案路徑
# MODEL_OUTPUT_DIR = os.path.join(OUTPUT_DIR, "bert_results")
# LOGGING_DIR = os.path.join(OUTPUT_DIR, "bert_logs")
# SAVED_MODEL_DIR = os.path.join(OUTPUT_DIR, "bert_model_chinese")
# PREDICTION_CSV_PATH = os.path.join(OUTPUT_DIR, "prediction_result.csv")
# PLOT_PATH = os.path.join(OUTPUT_DIR, "prediction_probability_distribution.png")


# # ===============================
# # ✅ 設定：快速訓練模式開關
# # ===============================
# QUICK_MODE = True  # True ➜ 抽樣快速訓練，False ➜ 全資料

# # ===============================
# # ✅ 載入資料（含欄位清理）
# # ===============================
# print("🔄 開始載入與處理資料...")
# try:
#     df1 = pd.read_csv("DATA_DIR/finfo_posts_產險_壽險_投資型.csv")
#     df2 = pd.read_csv("DATA_DIR/mobile01_處理後.csv")
#     df3 = pd.read_csv("DATA_DIR/ptt_語料_處理後.csv")
# except FileNotFoundError as e:
#     print(f"❌ 錯誤：找不到資料檔案！請確認路徑 '{DATA_DIR}' 下有指定的 CSV 檔案。")
#     print(f"詳細錯誤：{e}")
#     exit() # 找不到檔案直接結束程式

# for df in [df1, df2, df3]:
#     df.columns = df.columns.str.strip()

# def extract_text(row):
#     text = ''
#     # 優先使用 '發文內容'/'留言內容' 或 'content'
#     if 'content' in row and pd.notna(row['content']):
#          text += str(row['content']) + ' '
#     elif '發文內容' in row and pd.notna(row['發文內容']):
#         text += str(row['發文內容']) + ' '
#         # Finfos 可能同時有發文和留言，都加進去
#         if '留言內容' in row and pd.notna(row['留言內容']):
#            text += str(row['留言內容'])

#     # 如果上面都沒有，嘗試其他可能的欄位（根據實際資料調整）
#     # elif '其他欄位' in row and pd.notna(row['其他欄位']):
#     #     text += str(row['其他欄位'])

#     return text.strip()

# def prepare_labeled_df(df):
#     df['text'] = df.apply(extract_text, axis=1)
#     # 確保 '詐騙關鍵詞次數' 欄位存在且為數值，若不存在或非數值則預設為0
#     if '詐騙關鍵詞次數' not in df.columns:
#         print(f"⚠️ 警告：資料集中缺少 '詐騙關鍵詞次數' 欄位，將假設所有 label 為 0。")
#         df['label'] = 0
#     else:
#         # 嘗試轉換為數值，無法轉換的設為 0
#         df['詐騙關鍵詞次數'] = pd.to_numeric(df['詐騙關鍵詞次數'], errors='coerce').fillna(0)
#         df['label'] = df['詐騙關鍵詞次數'].apply(lambda x: 1 if x > 0 else 0)
#     return df[['text', 'label']]

# # 處理 df2 和 df3 作為訓練資料
# df_train_list = []
# print("處理 df2 (Mobile01)...")
# df_train_list.append(prepare_labeled_df(df2))
# print("處理 df3 (PTT)...")
# df_train_list.append(prepare_labeled_df(df3))
# df_train = pd.concat(df_train_list, ignore_index=True)

# # 處理 df1 作為預測資料
# print("處理 df1 (Finfo)...")
# df_unlabeled = df1.copy()
# df_unlabeled['text'] = df_unlabeled.apply(extract_text, axis=1)

# # 移除 text 為空字串的資料 (避免 Tokenizer 出錯)
# df_train = df_train[df_train['text'].str.strip() != '']
# df_unlabeled = df_unlabeled[df_unlabeled['text'].str.strip() != '']

# # 🔹 快速模式：抽樣訓練資料
# if QUICK_MODE:
#     n_samples = min(10000, len(df_train)) # 防止總資料少於一萬筆而出錯
#     if len(df_train) > n_samples:
#         df_train = df_train.sample(n=n_samples, random_state=42)
#         print(f"⚡ 快速模式啟用：抽樣 {n_samples} 筆資料訓練")
#     else:
#         print(f"⚡ 快速模式啟用：使用全部 {len(df_train)} 筆資料訓練 (總數少於或等於抽樣數)")


# print(f"✅ 清理與準備完成：")
# print(f"   訓練樣本數：{len(df_train)}")
# print(f"   預測樣本數：{len(df_unlabeled)}")
# print(f"   訓練資料標籤分佈:\n{df_train['label'].value_counts(normalize=True)}")


# # ===============================
# # ✅ tokenizer + 切分 Dataset
# # ===============================
# print("\n🔄 開始 Tokenization 與資料集準備...")
# # ... (tokenizer 載入, tokenize_function 定義, Dataset.from_pandas, map 等不變) ...

# try:
#     raw_dataset = Dataset.from_pandas(df_train)
#     # 移除原始 text 節省記憶體, 確保 'label' 欄位存在以便後續轉換
#     tokenized_dataset = raw_dataset.map(tokenize_function, batched=True, remove_columns=['text'])
#     tokenized_dataset = tokenized_dataset.rename_column("label", "labels") # 先改名
#     tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

#     # --- 👇👇👇 新增：轉換 labels 欄位為 ClassLabel 👇👇👇 ---
#     from datasets import ClassLabel, Features # 確保導入
#     print("🔄 Casting 'labels' column to ClassLabel for stratification...")
#     # 定義新的 Features，明確指定 'labels' 的類型
#     new_features = tokenized_dataset.features.copy()
#     new_features['labels'] = ClassLabel(num_classes=2, names=['not_scam', 'scam']) # num_classes=2 因為是二元分類
#     tokenized_dataset = tokenized_dataset.cast(new_features)
#     print(f"✅ 'labels' column type after casting: {tokenized_dataset.features['labels']}")
#     # --- 👆👆👆 新增結束 👆👆👆 ---


#     # 現在可以進行分層抽樣了
#     split_dataset = tokenized_dataset.train_test_split(
#         test_size=0.2,
#         seed=42,
#         stratify_by_column="labels" # 現在 labels 是 ClassLabel，可以分層了
#     )
#     train_dataset = split_dataset["train"]
#     eval_dataset = split_dataset["test"]

#     print(f"✅ Tokenization 與資料集切分完成：")
#     print(f"   Train size: {len(train_dataset)}")
#     print(f"   Eval size: {len(eval_dataset)}")
# except Exception as e:
#     print(f"❌ Tokenization 或資料集轉換/切分失敗：{e}")
#     # 打印更詳細的追蹤信息，有助於除錯
#     import traceback
#     traceback.print_exc()
#     exit()
# # ===============================
# # ✅ 模型 + 訓練設定 + EarlyStopping
# # ===============================
# print("\n🔄 開始設定模型與訓練參數...")
# model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

# # --- 訓練參數 ---
# # !! 注意 !! 如果遇到 CUDA Out of Memory (OOM) 錯誤，請優先降低 per_device_train_batch_size
# # GTX 1060 6GB 在 max_length=256 時，batch_size 可能需要設為 4 或 2
# # 如果 max_length=128，batch_size=8 或許可以，但仍需嘗試
# TRAIN_BATCH_SIZE = 4 # 降低 batch size 以適應 GTX 1060 (可嘗試 8, 4, 2)
# EVAL_BATCH_SIZE = 8 # 評估時 batch size 通常可以稍大 (可嘗試 16, 8)
# NUM_EPOCHS = 3 if QUICK_MODE else 10

# training_args = TrainingArguments(
#     output_dir=MODEL_OUTPUT_DIR,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=TRAIN_BATCH_SIZE,
#     per_device_eval_batch_size=EVAL_BATCH_SIZE,
#     num_train_epochs=NUM_EPOCHS,
#     weight_decay=0.01,
#     load_best_model_at_end=True,
#     logging_dir=LOGGING_DIR,
#     logging_strategy="epoch",
#     metric_for_best_model="eval_f1", # 改用 F1 作為主要評估指標可能更適合不平衡資料
#     save_total_limit=1,          # 只保留最好的模型
#     report_to="none",            # 關掉 wandb 等外部報告
#     fp16=torch.cuda.is_available(), # ✅ 自動啟用混合精度 (如果 CUDA 可用)
#     gradient_accumulation_steps=2, # 梯度累積，等效 batch size = TRAIN_BATCH_SIZE * 2，可稍微彌補小 batch size
#     learning_rate=3e-5,          # 常用的學習率
# )

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     preds = np.argmax(logits, axis=-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
#     acc = accuracy_score(labels, preds)
#     return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.001)] # 加入 threshold 避免微小改善也繼續等
# )

# print(f"✅ 設定完成，準備開始訓練...")
# print(f"   使用 GPU: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"   CUDA 裝置: {torch.cuda.get_device_name(0)}")
# print(f"   訓練 Epochs: {NUM_EPOCHS}")
# print(f"   訓練 Batch Size (Per Device): {TRAIN_BATCH_SIZE}")
# print(f"   梯度累積步數: {training_args.gradient_accumulation_steps}")
# print(f"   等效 Batch Size: {TRAIN_BATCH_SIZE * training_args.gradient_accumulation_steps}")
# print(f"   評估 Batch Size (Per Device): {EVAL_BATCH_SIZE}")
# print(f"   混合精度 (FP16): {training_args.fp16}")

# # ===============================
# # ✅ 模型訓練
# # ===============================
# print("\n🏋️‍♂️ 開始訓練模型...")
# try:
#     trainer.train()
#     print("✅ 訓練完成！")
# except RuntimeError as e:
#     if "CUDA out of memory" in str(e):
#         print("\n❌ 錯誤：CUDA Out of Memory！")
#         print("🧠 建議：請嘗試降低 TrainingArguments 中的 'per_device_train_batch_size'。")
#         print(f"   目前的設定是 {TRAIN_BATCH_SIZE}，可以試試 4 或 2。")
#         print(f"   或者減少 tokenize_function 中的 'max_length' (目前是 {tokenizer.model_max_length})。")
#     else:
#         print(f"\n❌ 訓練過程中發生錯誤：{e}")
#     exit()
# except Exception as e:
#     print(f"\n❌ 訓練過程中發生未預期錯誤：{e}")
#     exit()


# # ===============================
# # ✅ 儲存模型
# # ===============================
# print(f"\n💾 正在儲存最佳模型至：{SAVED_MODEL_DIR}")
# trainer.save_model(SAVED_MODEL_DIR)
# tokenizer.save_pretrained(SAVED_MODEL_DIR)
# print("✅ 模型與 Tokenizer 儲存完成。")

# # ===============================
# # ✅ 預測流程 + 輸出 CSV
# # ===============================
# print("\n📊 開始使用訓練好的模型進行預測...")

# # 重新載入最佳模型以確保使用的是儲存的版本
# model = BertForSequenceClassification.from_pretrained(SAVED_MODEL_DIR)
# tokenizer = BertTokenizerFast.from_pretrained(SAVED_MODEL_DIR)

# # 設定 pipeline device
# device_num = 0 if torch.cuda.is_available() else -1

# pipe = TextClassificationPipeline(
#     model=model,
#     tokenizer=tokenizer,
#     return_all_scores=True, # 獲取所有類別分數
#     device=device_num       # 使用 GPU (如果可用)
# )

# # 執行預測 (如果資料量大，這一步可能也需要一些時間)
# # 使用 tolist() 避免傳遞 Series 物件可能造成的問題
# texts_to_predict = df_unlabeled["text"].tolist()

# # 分批預測以控制記憶體使用
# batch_size = 32 # 可以根據你的 RAM 和 VRAM 調整
# results = []
# print(f"   預測總筆數: {len(texts_to_predict)}，批次大小: {batch_size}")
# for i in range(0, len(texts_to_predict), batch_size):
#     batch_texts = texts_to_predict[i:i + batch_size]
#     batch_results = pipe(batch_texts)
#     results.extend(batch_results)
#     print(f"   已預測 {min(i + batch_size, len(texts_to_predict))}/{len(texts_to_predict)} 筆")


# # 處理預測結果
# # 假設 Label 'LABEL_1' 代表詐騙 (需根據 pipeline 實際輸出確認)
# pred_labels = []
# pred_probs = []
# for res in results:
#     score_label_1 = 0.0
#     highest_score_label = 0
#     highest_score = 0.0
#     for item in res:
#         if item['label'] == 'LABEL_1':
#             score_label_1 = item['score']
#         if item['score'] > highest_score:
#             highest_score = item['score']
#             # 從 'LABEL_X' 中取出數字 X
#             highest_score_label = int(item['label'].split('_')[-1])

#     pred_labels.append(highest_score_label)
#     pred_probs.append(round(score_label_1, 4)) # 儲存 LABEL_1 的機率

# df_unlabeled["predicted_label"] = pred_labels
# df_unlabeled["predicted_prob_scam"] = pred_probs # 欄位名改為更清晰

# # 選擇要輸出的欄位 (只保留必要欄位，避免檔案過大)
# output_columns = ['text', 'predicted_label', 'predicted_prob_scam']
# # 如果原始 df1 有 ID 或其他重要欄位，也可以加入
# # if 'id_column' in df_unlabeled.columns:
# #     output_columns.insert(0, 'id_column')

# df_unlabeled[output_columns].to_csv(PREDICTION_CSV_PATH, index=False, encoding="utf-8-sig")

# print(f"✅ 完成預測！結果儲存於：{PREDICTION_CSV_PATH}")


# # ===============================
# # ✅ (新增) 繪製預測機率分佈圖
# # ===============================
# print("\n🎨 正在繪製預測機率分佈圖...")

# # --- 字體設定 (針對本機 Matplotlib) ---
# # Windows: C:/Windows/Fonts/msjh.ttc (微軟正黑體)
# # macOS: /System/Library/Fonts/STHeiti Medium.ttc (黑體-繁 中等)
# # Linux: 可能需要先安裝字型 (如 Noto Sans CJK TC) 並找到 .ttf 或 .otf 檔
# # 如果找不到字體或不確定，可以註解掉下面幾行，但中文可能顯示為方塊
# try:
#     # 嘗試設定中文字體 (請根據你的作業系統修改路徑或字體名稱)
#     font_path = None
#     if os.path.exists("C:/Windows/Fonts/msjh.ttc"):
#         font_path = "C:/Windows/Fonts/msjh.ttc"
#         font_prop = fm.FontProperties(fname=font_path)
#         plt.rcParams['font.family'] = font_prop.get_name()
#     elif os.path.exists("/System/Library/Fonts/STHeiti Medium.ttc"):
#          # macOS 可能可以直接設定字體名稱
#          plt.rcParams['font.sans-serif'] = ['Heiti TC'] # 或 'PingFang TC'
#     elif os.path.exists("/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"): # 範例 Linux 路徑
#         font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
#         font_prop = fm.FontProperties(fname=font_path)
#         plt.rcParams['font.family'] = font_prop.get_name()
#     else:
#         print("⚠️ 未找到指定的中文字體，圖表中的中文可能無法正常顯示。")

#     plt.rcParams['axes.unicode_minus'] = False # 解決負號顯示問題
# except Exception as font_e:
#     print(f"⚠️ 設定中文字體時發生錯誤: {font_e}. 中文可能無法正常顯示。")
# # --- 字體設定結束 ---

# plt.figure(figsize=(12, 7))
# sns.histplot(data=df_unlabeled, x="predicted_prob_scam", kde=True, bins=50, color="skyblue") # 使用 50 個區間
# plt.title("預測為詐騙 (Label=1) 的機率分佈", fontsize=16)
# plt.xlabel("模型預測為詐騙的機率值", fontsize=12)
# plt.ylabel("資料筆數", fontsize=12)
# plt.grid(axis='y', linestyle='--', alpha=0.7)

# # 加上閾值線
# threshold = 0.5
# plt.axvline(x=threshold, color='r', linestyle='--', label=f'分類閾值 = {threshold}')

# # 在圖上顯示一些統計數據
# mean_prob = df_unlabeled["predicted_prob_scam"].mean()
# median_prob = df_unlabeled["predicted_prob_scam"].median()
# plt.axvline(x=mean_prob, color='g', linestyle=':', label=f'平均機率 = {mean_prob:.3f}')
# plt.legend(fontsize=10)

# plt.tight_layout() # 自動調整邊距

# # 儲存圖片
# plt.savefig(PLOT_PATH, dpi=300) # 提高解析度
# # plt.show() # 如果希望在 VS Code 中彈出視窗顯示圖表，取消此行註解

# print(f"✅ 機率分佈圖儲存於：{PLOT_PATH}")


# # ===============================
# # ✅ (新增) 簡單解釋
# # ===============================
# print("\n===== 📊 圖表與結果解釋 =====")
# print(f"模型已對 {len(df_unlabeled)} 筆來自 '{os.path.basename(FILE_DF1)}' 的資料進行了預測。")
# print(f"詳細預測結果（含原始文本、預測標籤、詐騙機率）儲存在：\n{PREDICTION_CSV_PATH}")
# print(f"\n預測機率分佈圖儲存在：\n{PLOT_PATH}")
# print("這張圖顯示了模型將每筆資料判斷為「詐騙」(Label=1) 的信心程度（機率值）的分佈情況。")
# print("\n圖表觀察重點：")
# print("1. 【形狀】機率分佈是否呈現『雙峰』？")
# print("   - 若大量資料的機率集中在接近 0 和接近 1 的兩端，表示模型對大部分樣本的判斷比較『有信心』（明確判斷為非詐騙或詐騙）。")
# print("   - 若機率集中在中間（如 0.5 附近），表示模型對很多樣本的判斷比較『模糊』或『不確定』。")
# print("2. 【重心】機率主要集中在哪一邊？")
# print(f"   - 平均機率 ({mean_prob:.3f}) 和中位數機率 ({median_prob:.3f}) 可以提供參考。")
# print("   - 如果大部分機率值都偏低（接近 0），表示模型傾向於認為大部分文本『不是』詐騙。")
# print("   - 如果大部分機率值都偏高（接近 1），表示模型傾向於認為大部分文本『是』詐騙。")
# print("3. 【閾值附近】紅色虛線（預設 0.5）是常用的分類界線。")
# print("   - 在這條線附近的樣本，是模型判斷結果『模棱兩可』的區域。改變閾值會影響最終哪些樣本被歸類為詐騙。")
# print("   - 這些閾值附近的樣本可能最需要人工複查。")
# print("\n👉 建議：打開 CSV 檔案，對照高機率、低機率以及機率在閾值附近的文本，檢查模型的判斷是否符合預期。")
# print("==============================")

In [2]:
# 重新載入所有必要的檔案與套件（包括 jieba）
!pip install jieba --quiet

import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 檔案路徑
file_paths = {
    "mobile01": "DATA_DIR/mobile01_處理後.csv",
    "ptt": "DATA_DIR/ptt_語料_處理後.csv",
    "finfo": "DATA_DIR/finfo_posts_產險_壽險_投資型.csv",
    "keywords": "DATA_DIR/500精簡詐騙字詞_UTF8.csv"
}

# 載入資料
dfs = {name: pd.read_csv(path) for name, path in file_paths.items()}

# 合併 PTT 與 Mobile01 資料
ptt_df = dfs["ptt"]
mobile01_df = dfs["mobile01"]
combined_df = pd.concat([ptt_df, mobile01_df], ignore_index=True)

# 載入詐騙關鍵字
keywords_df = dfs["keywords"]
scam_keywords = keywords_df["詞語"].dropna().tolist()

# 結合欄位文本
def combine_text(row):
    content = ''
    for col in ['發文內容', '留言內容', 'content']:
        if col in row and pd.notna(row[col]):
            content += str(row[col]) + ' '
    return content.strip()

combined_df['text'] = combined_df.apply(combine_text, axis=1)

# 標記是否為詐騙句子
def is_scam(text):
    return int(any(kw in text for kw in scam_keywords))

combined_df['label'] = combined_df['text'].apply(is_scam)

# 篩除空文字
combined_df = combined_df[combined_df['text'].str.strip().astype(bool)]

# 中文斷詞
def tokenize(text):
    return " ".join(jieba.cut(text))

combined_df['text_tokenized'] = combined_df['text'].apply(tokenize)

# 拆分資料
X_train, X_test, y_train, y_test = train_test_split(
    combined_df['text_tokenized'],
    combined_df['label'],
    test_size=0.2,
    random_state=42
)

# Tokenizer 編碼文字
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# 補齊長度
max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# 建立簡單 LSTM 模型
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練模型（縮短為 3 epoch）
model.fit(X_train_pad, y_train, epochs=3, batch_size=64, validation_split=0.2)

# 定義詐騙預測函數
def predict_scam(sentence):
    tokenized = " ".join(jieba.cut(sentence))
    seq = tokenizer.texts_to_sequences([tokenized])
    padded = pad_sequences(seq, maxlen=max_len)
    prob = model.predict(padded)[0][0]
    return prob

# 範例預測
example_sentence = "您中獎了！請立即回撥專線領獎"
example_prob = predict_scam(example_sentence)
example_prob


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ROBBY1~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.797 seconds.
Prefix dict has been built successfully.


Epoch 1/3


c:\Users\Robby1206\anaconda3\envs\web_scraping\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


953/953 ━━━━━━━━━━━━━━━━━━━━ 53s 52ms/step - accuracy: 0.8636 - loss: 0.3040 - val_accuracy: 0.9562 - val_loss: 0.1241
Epoch 2/3
953/953 ━━━━━━━━━━━━━━━━━━━━ 50s 53ms/step - accuracy: 0.9636 - loss: 0.1051 - val_accuracy: 0.9585 - val_loss: 0.1232
Epoch 3/3
953/953 ━━━━━━━━━━━━━━━━━━━━ 52s 54ms/step - accuracy: 0.9685 - loss: 0.0939 - val_accuracy: 0.9567 - val_loss: 0.1257
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 610ms/step


0.9780651

In [3]:
# 定義詐騙預測函數
def predict_scam(sentence):
    tokenized = " ".join(jieba.cut(sentence))
    seq = tokenizer.texts_to_sequences([tokenized])
    padded = pad_sequences(seq, maxlen=max_len)
    prob = model.predict(padded)[0][0]
    return prob

# 範例預測
example_sentence = "	您的帳戶異常，需立即操作ATM解除分期付款。"
example_prob = predict_scam(example_sentence)
example_prob

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


0.9997644

In [9]:
# ========== 1. 安裝套件（第一次用） ==========
# pip install jieba pandas scikit-learn tensorflow openpyxl

# ========== 2. 載入必要模組 ==========
import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# ========== 3. 讀取資料 ==========
ptt_df = pd.read_csv("DATA_DIR/ptt_語料_處理後.csv")
mobile_df = pd.read_csv("DATA_DIR/mobile01_處理後.csv")
keywords_df = pd.read_csv("DATA_DIR/500精簡詐騙字詞_UTF8.csv")
scam_keywords = keywords_df["詞語"].dropna().tolist()

df = pd.concat([ptt_df, mobile_df], ignore_index=True)

# 合併內容欄
def combine_text(row):
    for col in ['發文內容', '留言內容', 'content']:
        if col in row and pd.notna(row[col]):
            return str(row[col])
    return ""

df["text"] = df.apply(combine_text, axis=1)

# 標記詐騙與否
df["label"] = df["text"].apply(lambda x: int(any(k in x for k in scam_keywords)))
df = df[df["text"].str.strip().astype(bool)]

# 去除關鍵字，避免資料洩漏
def remove_keywords(text, keywords):
    for k in keywords:
        text = text.replace(k, "")
    return text

df["text_clean"] = df["text"].apply(lambda x: remove_keywords(x, scam_keywords))
df["text_tokenized"] = df["text_clean"].apply(lambda x: " ".join(jieba.cut(x)))

# ========== 4. 建立訓練集 ==========
X_train, X_test, y_train, y_test = train_test_split(df["text_tokenized"], df["label"], test_size=0.2, random_state=42)

# ========== 5. 向量化 ==========
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# ========== 6. 建立模型 ==========
model = Sequential([
    Embedding(5000, 64, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# ========== 7. 訓練模型 ==========
model.fit(X_train_pad, y_train, epochs=3, batch_size=64, validation_split=0.2)

# ========== 8. 預測函式 ==========
def predict_scam(sentence):
    tokenized = " ".join(jieba.cut(sentence))
    seq = tokenizer.texts_to_sequences([tokenized])
    padded = pad_sequences(seq, maxlen=max_len)
    prob = model.predict(padded)[0][0]
    return prob

# 測試例句
print(predict_scam("您中獎了！請立即回撥專線領獎"))


Epoch 1/3


c:\Users\Robby1206\anaconda3\envs\web_scraping\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


953/953 ━━━━━━━━━━━━━━━━━━━━ 53s 52ms/step - accuracy: 0.7586 - loss: 0.4747 - val_accuracy: 0.7833 - val_loss: 0.4257
Epoch 2/3
953/953 ━━━━━━━━━━━━━━━━━━━━ 50s 52ms/step - accuracy: 0.8041 - loss: 0.4001 - val_accuracy: 0.7863 - val_loss: 0.4202
Epoch 3/3
953/953 ━━━━━━━━━━━━━━━━━━━━ 50s 53ms/step - accuracy: 0.8174 - loss: 0.3747 - val_accuracy: 0.7882 - val_loss: 0.4246
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
0.53160834


In [10]:
import pandas as pd
import jieba
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 載入 Excel
df = pd.read_excel("語句預測.xlsx")
df.columns = df.columns.str.strip()

# 自動判斷句子欄位名稱
if "句子" in df.columns:
    sentence_col = "句子"
else:
    sentence_col = df.columns[0]  # 預設使用第一欄

# 取得語句列表
sentences = df[sentence_col].dropna().astype(str).tolist()

# 預測函數
def predict_scam_batch(sent_list):
    results = []
    for sentence in sent_list:
        tokenized = " ".join(jieba.cut(sentence))
        seq = tokenizer.texts_to_sequences([tokenized])
        padded = pad_sequences(seq, maxlen=max_len)
        prob = model.predict(padded)[0][0]
        results.append(prob)
    return results

# 預測
df["詐騙機率"] = predict_scam_batch(sentences)

# 顯示結果
print(df[[sentence_col, "詐騙機率"]].head(10))

# 儲存
df.to_excel("語句詐騙預測結果.xlsx", index=False)
print("✅ 已儲存為 語句詐騙預測結果.xlsx")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━